In [26]:
import NO_parser
import NV_route
import pandas as pd
from pathlib import Path

In [27]:
'''Constants to create dummy data'''
directory_string = "C:\\Users\\msn\\OneDrive - Never Gray\\Software Development\\Next-Vis\\Projects and Issues\\2021-09-14 Tunnel Stencil\\Parsing\\"
file_name = "Form4.prn"
path_string = directory_string + file_name
file_path = Path(path_string)
visio_template = "TS_005.vsdx"
visio_template_folder = "C:/Users/msn/OneDrive - Never Gray/Software Development/Next-Vis/Projects and Issues/2021-09-14 Tunnel Stencil"
settings = {
    "ses_output_str": [file_path],
    "visio_template": "/".join([visio_template_folder, visio_template]),
    "results_folder_str": "",
    "simtime": 25.0,
    "version": "tbd",
    "control": "First",
    "output": ["Visio","visio_2_pdf","visio_2_png","visio_2_svg"],
    }
data, output_meta_data = NO_parser.parse_file(file_path)
simtime = 25.0

Parser msg: Read data from Form4.prn


In [67]:
'''Create column with True if fire is active'''
if 'form4_df' in output_meta_data:
    form4_df = output_meta_data['form4_df']
    form4_df['active_fire']= (form4_df['source_active'] <= simtime) & (form4_df['source_inactive'] >= simtime)
    # Create smaller truth table with only True segments and sub-segments
    form4_truth_table_sub = form4_df[form4_df['active_fire']==True]['active_fire']
    # Create smaller truth table using only segments
    grouped = form4_truth_table_sub.groupby(level=0)
    form4_truth_table_segment = grouped.first()
else:
    form4_truth_table_segment = None

In [68]:
'''Determine if Train is present in Section or not''' 
if ('TRA' in data) and ('form9_df' in output_meta_data):
    simtime_tra = data['TRA'].loc[simtime]
    # Create default empty series
    segments_with_trains_series = pd.Series(dtype='int64')
    for index, row in simtime_tra.iterrows():
        train_number = index
        route = row['Route_Number']
        train_front = row['Location']
        train_type_number = row['Train_Type_Number']
        length = output_meta_data['form9_df'].loc[train_type_number,'train_length']
        train_back = train_front + length
        df = output_meta_data['form_8f'].loc[route]
        criteria_train_front = (df['Start']<=train_front) & (df['End']>=train_front)
        criteria_train_middle = (df['Start']>=train_front) & (df['End']<=train_back)
        criteria_train_back = (df['Start']<=train_back) & (df['End']>=train_back)
        new_segments_series = df[criteria_train_front | criteria_train_middle | criteria_train_back].index.to_series().abs()
        if new_segments_series is not None:
            segments_with_trains_series= segments_with_trains_series.append(new_segments_series,ignore_index=True)
    segments_with_trains = segments_with_trains_series.unique().tolist()
    if len(segments_with_trains) > 0:
        train_false_table_segment = pd.Series(index = segments_with_trains, dtype = bool, name="train_present").fillna(value=True)
        train_truth_table_segment = ~train_false_table_segment
    else:
        train_truth_table_segment = None
else:
    train_truth_table_segment = None

In [69]:
#Get airflow values at simtime
simtime_df = data['SSA'].loc[(simtime)]
if form4_truth_table_segment is not None:
    simtime_df = simtime_df.join(form4_truth_table_segment, how='left')
else:
    simtime_df["active_fire"] = False
if train_truth_table_segment is not None:
    simtime_df = simtime_df.join(train_truth_table_segment, how="left")
else:
    simtime_df["train_present"] = False
fillin_values = {"active_fire":False, "train_present":False}
simtime_df.fillna(value=fillin_values, inplace=True)


ID               25.0_212
Title                 New
Airflow           36.0356
Air_Velocity        371.5
active_fire          True
train_present        True
Name: 212, dtype: object

In [70]:
simtime_df

,ID,Title,Airflow,Air_Velocity,active_fire,train_present
Segment,,,,,,
111,25.0_111,Saccardo,41.2529,425.3,False,False
112,25.0_112,Saccardo,41.2529,425.3,False,False
113,25.0_113,New,41.2529,425.3,False,False
114,25.0_114,New,41.2529,425.3,False,False
115,25.0_115,New,41.2529,425.3,False,False
...,...,...,...,...,...,...
930,25.0_930,New,-0.1310,-43.7,False,False
931,25.0_931,New,-25.4493,-63.6,False,False
935,25.0_935,New,-0.1310,-43.7,False,False
